In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

The following URL returns a JSON file with the University rankink already parsed:

In [3]:
SITE = "https://www.topuniversities.com"
URL = SITE+"/sites/default/files/qs-rankings-data/357051.txt"

In [4]:
N = 10
r = requests.get(URL)
all_data = r.json()
print(list(all_data.keys()))
data = all_data.get("data")[:N]
print(list(data[0].keys()))

['data']
['guide', 'nid', 'score', 'core_id', 'url', 'country', 'cc', 'title', 'stars', 'rank_display', 'logo', 'region']


In [5]:
s = requests.Session()
reqs = [(req_id, s.prepare_request(requests.Request('GET', SITE+entry["url"]))) for req_id, entry in enumerate(data)]
print(len(reqs), "requests to be sent.")

resps = [(req_id, s.send(req)) for req_id, req in reqs]

done = [(req_id, resp.text) for req_id, resp in resps if resp.status_code == 200]
failed = [(req_id, resp) for req_id, resp in resps if resp.status_code != 200]

print("%d done, %d failed." % (len(done), len(failed)))

10 requests to be sent.
10 done, 0 failed.


In [6]:
import re
non_digit = re.compile('[^0-9]')

NEW_COLS = ("fac_c_total", "fac_c_inter", "stu_c_total", "stu_c_inter")

from multiprocessing import Pool, cpu_count

def resp_to_counts(req):
    req_id, resp = req
    page = BeautifulSoup(resp, "html.parser")
    div = page.body.find("div", class_="view-academic-data-profile")
    fac_counts = [int(re.sub(non_digit,'', tot.string)) for tot in div.find_all("div", class_="number")]
    return req_id, dict(zip(NEW_COLS, fac_counts))

print("Parsing responses using up to %d threads..." % cpu_count()) 
with Pool(cpu_count()) as p:
    for req_id, counts in p.map(resp_to_counts, done):
        data[req_id].update(counts)

Parsing responses using up to 4 threads...


In [7]:
uni = pd.DataFrame(data, columns=["title", "rank_display", "country", "region"]+list(NEW_COLS)+["url"])
uni.rename(columns={"title":"name", "rank_display": "rank"},inplace=True)
# Convert the rank to a numerical type
uni["rank"] = uni["rank"].str.extract('(\d+)', expand=False).astype(int)
uni.head()

,name,rank,country,region,fac_c_total,fac_c_inter,stu_c_total,stu_c_inter,url
0,Massachusetts Institute of Technology (MIT),1,United States,North America,2982,1679,11067,3717,/universities/massachusetts-institute-technolo...
1,Stanford University,2,United States,North America,4285,2042,15878,3611,/universities/stanford-university
2,Harvard University,3,United States,North America,4350,1311,22429,5266,/universities/harvard-university
3,California Institute of Technology (Caltech),4,United States,North America,953,350,2255,647,/universities/california-institute-technology-...
4,University of Cambridge,5,United Kingdom,Europe,5490,2278,18770,6699,/universities/university-cambridge


In [8]:
uni.describe()

,rank,fac_c_total,fac_c_inter,stu_c_total,stu_c_inter
count,10.00000,10.000000,10.000000,10.000000,10.000000
mean,5.50000,4001.100000,1777.000000,17066.100000,6183.500000
std,3.02765,1841.976622,816.462151,7547.545745,3906.429238
min,1.00000,953.000000,350.000000,2255.000000,647.000000
25%,3.25000,2603.250000,1403.000000,14137.250000,3637.500000
50%,5.50000,4107.500000,1964.000000,17430.000000,5982.500000
75%,7.75000,5205.000000,2226.250000,19791.250000,7510.500000
max,10.00000,6750.000000,2964.000000,31080.000000,14854.000000
